https://learn.adafruit.com/circuitpython-basics-i2c-and-spi/i2c-devices

https://learn.adafruit.com/circuitpython-digital-inputs-and-outputs/digital-outputs

https://learn.adafruit.com/thermistor/circuitpython

In [17]:
import board
import busio
import time
import numpy as np
import adafruit_character_lcd.character_lcd_spi as character_lcd

No connection to CiruitPython VM: found no board

In [ ]:
spi = busio.SPI(board.SCK, MOSI=board.MOSI)
latch = digitalio.DigitalInOut(board.D5)
cols = 16
rows = 2
lcd = character_lcd.Character_LCD_SPI(spi, latch, cols, rows)

In [4]:
def bitstring_to_bytes(s):
    return int(s, 2).to_bytes((len(s) + 7) // 8, byteorder='big')

No connection to CiruitPython VM: found no board

In [3]:
i2c = busio.I2C(board.SCL, board.SDA)
i2c_2 = busio.I2C(board.D9, board.D6)

No connection to CiruitPython VM: found no board

In [4]:
while not i2c.try_lock():
    pass

No connection to CiruitPython VM: found no board

In [1]:
a = [hex(x) for x in i2c.scan()]

No connection to CiruitPython VM: found no board

In [1]:
enable = 1
frequency = 3
wavetype = 1


frequency_opt = ["00", "01", "10", "11"]
mc1_address = ""
mc2_address = ""
mc_cmd = [bitstring_to_bytes(str(enable) + frequency_opt[frequency] + str(wavetype))]

No connection to CiruitPython VM: found no board

In [3]:
i2c.writeto(mc1_address, bytes([0x3F00, mc_cmd]), stop=False)
i2c_2.writeto(mc2_address, bytes([0x3F00, mc_cmd]), stop=False)

No connection to CiruitPython VM: found no board

In [14]:
rheostat_adress = hex(101100)
wiper0_address = 0x00
wiper1_address = 0x01
increment = 0x01
decrement = 0x02

No connection to CiruitPython VM: found no board

In [7]:
coil1_address = 0x00
coil2_address = 0x00

No connection to CiruitPython VM: found no board

In [21]:
def current1(coil, write_cmd="", read_cmd=""):
    i2c.writeto(coil, bytes(write_cmd), stop=False)
    data = bytearray(read_cmd)
    i2c.readfrom_into(coil, data)
    value = data[0] << 8 | data[1]
    I = (value & 0xFFF) / 16.0
    if value & 0x1000:
        I -= 256.0
    return I

def current2(coil, write_cmd="", read_cmd=""):
    i2c_2.writeto(coil, bytes(write_cmd), stop=False)
    data = bytearray(read_cmd)
    i2c_2.readfrom_into(coil, data)
    value = data[0] << 8 | data[1]
    I = (value & 0xFFF) / 16.0
    if value & 0x1000:
        I -= 256.0
    return I

No connection to CiruitPython VM: found no board

In [6]:
"""
while abs(current(coil1_address)) <= 0.1:
    i2c.writeto(mosfet1_address, bytes(mosfet_ON_cmd), stop=False)
    time.sleep(0.5)
    
while abs(current(coil2_address)) <= 0.1:
    i2c.writeto(mosfet2_address, bytes(mosfet_ON_cmd), stop=False)
    time.sleep(0.5)
"""

No connection to CiruitPython VM: found no board

In [4]:
adjust_current_c1 = True
while adjust_current_c1:
    
    currents_c1 = []
    t = 0
    while t < 100:
        currents_c1.append(current1(coil1_address))
        time.sleep(0.001)
        t += 1
    
    if np.average(currents_c1) > 15:
        enable = 0
        frequency = 0
        mc_cmd = [bitstring_to_bytes(str(enable) + frequency_opt[frequency] + str(wavetype))]
        i2c.writeto(mc_address, bytes([0x3F00, mc_cmd]), stop=False)
        a = 0
        while a < 100:
            i2c.writeto(rheostat_adress, bytes([wiper0_address, increment]), stop=False)
            i2c.writeto(rheostat_adress, bytes([wiper1_address, increment]), stop=False)
            a+=1
        
    elif np.average(currents_c1) < 9.5:
        i2c.writeto(rheostat_adress, bytes([wiper0_address, decrement]), stop=False)
        i2c.writeto(rheostat_adress, bytes([wiper1_address, decrement]), stop=False)
    elif np.average(currents_c1) > 11:
        i2c.writeto(rheostat_adress, bytes([wiper0_address, increment]), stop=False)
        i2c.writeto(rheostat_adress, bytes([wiper1_address, increment]), stop=False)
    else:
        adjust_current_c1 = False
    
adjust_current_c2 = True
while adjust_current_c2:
    
    currents_c2 = []
    t = 0
    while t < 10:
        currents_c2.append(current2(coil2_address))
        time.sleep(0.01)
        t += 1
    
    if np.average(currents_c2) > 2:
        enable = 0
        frequency = 0
        mc_cmd = [bitstring_to_bytes(str(enable) + frequency_opt[frequency] + str(wavetype))]
        i2c_2.writeto(mc_address, bytes([0x3F00, mc_cmd]), stop=False)
        a = 0
        while a < 100:
            i2c_2.writeto(rheostat_adress, bytes([wiper0_address, increment]), stop=False)
            i2c_2.writeto(rheostat_adress, bytes([wiper1_address, increment]), stop=False)
            a+=1
        
    elif np.average(currents_c2) < 0.8:
        i2c_2.writeto(rheostat_adress, bytes([wiper0_address, decrement]), stop=False)
        i2c_2.writeto(rheostat_adress, bytes([wiper1_address, decrement]), stop=False)
    elif np.average(currents_c2) > 1.2:
        i2c_2.writeto(rheostat_adress, bytes([wiper0_address, decrement]), stop=False)
        i2c_2.writeto(rheostat_adress, bytes([wiper1_address, increment]), stop=False)
    else:
        adjust_current_c2 = False

No connection to CiruitPython VM: found no board

In [5]:
i2c.unlock()

No connection to CiruitPython VM: found no board